# LangGraphのcheckpointのサンプルコード

In [1]:
import os
import operator
from typing import Annotated, Any
from langchain_core.messages import SystemMessage, HumanMessage, BaseMessage
from langchain_openai import ChatOpenAI
from pydantic import BaseModel, Field
from langfuse.callback import CallbackHandler

In [2]:
OPENROUTER_API_KEY = os.getenv("OPENROUTER_API_KEY")
if OPENROUTER_API_KEY is None:
    raise ValueError("OPENROUTER_API_KEY environment variable is not set.")

OPENROUTER_BASE_URL = "https://openrouter.ai/api/v1"

In [3]:
LANGFUSE_HOST = os.getenv("LANGFUSE_HOST")
SECRET_KEY = os.getenv("SECRET_KEY")
PUBLIC_KEY = os.getenv("PUBLIC_KEY")

langfuse_handler = CallbackHandler(
    public_key=PUBLIC_KEY,
    secret_key=SECRET_KEY,
    host=LANGFUSE_HOST,
)


In [4]:
# グラフのステートを定義
class State(BaseModel):
    query: str
    messages: Annotated[list[BaseMessage], operator.add] = Field(default=[])

In [5]:
# メッセージを追加するノード関数
def add_message(state: State) -> dict[str, Any]:
    additional_messages = []

    if not state.messages:
        additional_messages.append(SystemMessage(content="あなたは最小限の応答をする対話エージェントです。"))
    additional_messages.append(HumanMessage(content=state.query))
    return {"messages": additional_messages}

In [6]:
# LLMからの応答を追加するノード関数
def llm_response(state: State) -> dict[str, Any]:
    llm = ChatOpenAI(
        openai_api_key=OPENROUTER_API_KEY,
        openai_api_base=OPENROUTER_BASE_URL,
        model_name="google/gemma-3-27b-it:free",
    )
    ai_message = llm.invoke(state.messages, config={"callbacks": [langfuse_handler]})
    return {"messages": [ai_message]}

In [ ]:
from pprint import pprint
from langchain_core.runnables import RunnableConfig
from langgraph.checkpoint.base import BaseCheckpointSaver


def print_checkpoint_dump(checkpointer: BaseCheckpointSaver, config: RunnableConfig):
    checkpoint_tuple = checkpointer.get_tuple(config)

    print("チェックポイントデータ:")
    pprint(checkpoint_tuple.checkpoint)
    print("\nメタデータ:")
    pprint(checkpoint_tuple.metadata)

In [8]:
from langgraph.graph import StateGraph, END
from langgraph.checkpoint.memory import MemorySaver

In [9]:
# グラフを設定
graph = StateGraph(State)
graph.add_node("add_message", add_message)
graph.add_node("llm_response", llm_response)

graph.set_entry_point("add_message")
graph.add_edge("add_message", "llm_response")
graph.add_edge("llm_response", END)

# チェックポインターを設定
checkpointer = MemorySaver()

# グラフをコンパイル
compiled_graph = graph.compile(checkpointer=checkpointer)

In [10]:
config = {"configurable": {"thread_id": "example-1"}, "callbacks": [langfuse_handler]}
user_query = State(query="私の好きなものはずんだ餅です。覚えておいてね。")
first_response = compiled_graph.invoke(user_query, config)
first_response

{'query': '私の好きなものはずんだ餅です。覚えておいてね。',
 'messages': [SystemMessage(content='あなたは最小限の応答をする対話エージェントです。', additional_kwargs={}, response_metadata={}),
  HumanMessage(content='私の好きなものはずんだ餅です。覚えておいてね。', additional_kwargs={}, response_metadata={}),
  AIMessage(content='\n了解しました。', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 5, 'prompt_tokens': 40, 'total_tokens': 45, 'completion_tokens_details': None, 'prompt_tokens_details': None}, 'model_name': 'google/gemma-3-27b-it', 'system_fingerprint': None, 'id': 'gen-1745217510-nosKfuWgYCMj1aZSmKVz', 'finish_reason': 'stop', 'logprobs': None}, id='run-c7e7a504-05b3-483a-b34b-ed569d156ee8-0', usage_metadata={'input_tokens': 40, 'output_tokens': 5, 'total_tokens': 45, 'input_token_details': {}, 'output_token_details': {}})]}

In [11]:
for checkpoint in checkpointer.list(config):
    print(checkpoint)

CheckpointTuple(config={'configurable': {'thread_id': 'example-1', 'checkpoint_ns': '', 'checkpoint_id': '1f01e7b3-e4da-6d4e-8002-6a0f246f4480'}}, checkpoint={'v': 2, 'ts': '2025-04-21T06:38:32.357389+00:00', 'id': '1f01e7b3-e4da-6d4e-8002-6a0f246f4480', 'channel_versions': {'__start__': '00000000000000000000000000000002.0.7118226758080944', 'query': '00000000000000000000000000000002.0.3590759194654277', 'messages': '00000000000000000000000000000004.0.13804351538684068', 'start:add_message': '00000000000000000000000000000003.0.9235249002318867', 'add_message': '00000000000000000000000000000004.0.011906834728009685', 'llm_response': '00000000000000000000000000000004.0.23668096507428016'}, 'versions_seen': {'__input__': {}, '__start__': {'__start__': '00000000000000000000000000000001.0.8573990836247918'}, 'add_message': {'start:add_message': '00000000000000000000000000000002.0.01227794035522567'}, 'llm_response': {'add_message': '00000000000000000000000000000003.0.5502931316410851'}}, 'c

In [12]:
print_checkpoint_dump(checkpointer, config)

チェックポイントデータ:
{'channel_values': {'llm_response': 'llm_response',
                    'messages': [SystemMessage(content='あなたは最小限の応答をする対話エージェントです。', additional_kwargs={}, response_metadata={}),
                                 HumanMessage(content='私の好きなものはずんだ餅です。覚えておいてね。', additional_kwargs={}, response_metadata={}),
                                 AIMessage(content='\n了解しました。', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 5, 'prompt_tokens': 40, 'total_tokens': 45, 'completion_tokens_details': None, 'prompt_tokens_details': None}, 'model_name': 'google/gemma-3-27b-it', 'system_fingerprint': None, 'id': 'gen-1745217510-nosKfuWgYCMj1aZSmKVz', 'finish_reason': 'stop', 'logprobs': None}, id='run-c7e7a504-05b3-483a-b34b-ed569d156ee8-0', usage_metadata={'input_tokens': 40, 'output_tokens': 5, 'total_tokens': 45, 'input_token_details': {}, 'output_token_details': {}})],
                    'query': '私の好きなものはずんだ餅です。覚えておいてね。'},
 'channel_version

In [13]:
user_query = State(query="私の好物は何か覚えてる？")
second_response = compiled_graph.invoke(user_query, config)
second_response

{'query': '私の好物は何か覚えてる？',
 'messages': [SystemMessage(content='あなたは最小限の応答をする対話エージェントです。', additional_kwargs={}, response_metadata={}),
  HumanMessage(content='私の好きなものはずんだ餅です。覚えておいてね。', additional_kwargs={}, response_metadata={}),
  AIMessage(content='\n了解しました。', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 5, 'prompt_tokens': 40, 'total_tokens': 45, 'completion_tokens_details': None, 'prompt_tokens_details': None}, 'model_name': 'google/gemma-3-27b-it', 'system_fingerprint': None, 'id': 'gen-1745217510-nosKfuWgYCMj1aZSmKVz', 'finish_reason': 'stop', 'logprobs': None}, id='run-c7e7a504-05b3-483a-b34b-ed569d156ee8-0', usage_metadata={'input_tokens': 40, 'output_tokens': 5, 'total_tokens': 45, 'input_token_details': {}, 'output_token_details': {}}),
  HumanMessage(content='私の好物は何か覚えてる？', additional_kwargs={}, response_metadata={}),
  AIMessage(content='\nずんだ餅。', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'complet

In [14]:
for checkpoint in checkpointer.list(config):
    print(checkpoint)

CheckpointTuple(config={'configurable': {'thread_id': 'example-1', 'checkpoint_ns': '', 'checkpoint_id': '1f01e7b4-6cb0-647c-8006-6a77f3d62036'}}, checkpoint={'v': 2, 'ts': '2025-04-21T06:38:46.600570+00:00', 'id': '1f01e7b4-6cb0-647c-8006-6a77f3d62036', 'channel_versions': {'__start__': '00000000000000000000000000000006.0.8812221959059374', 'query': '00000000000000000000000000000006.0.2068364970194544', 'messages': '00000000000000000000000000000008.0.6491719237330902', 'start:add_message': '00000000000000000000000000000007.0.3309745091226802', 'add_message': '00000000000000000000000000000008.0.9480927303271989', 'llm_response': '00000000000000000000000000000008.0.4328003221771848'}, 'versions_seen': {'__input__': {}, '__start__': {'__start__': '00000000000000000000000000000005.0.05459234375282418'}, 'add_message': {'start:add_message': '00000000000000000000000000000006.0.21340678623293463'}, 'llm_response': {'add_message': '00000000000000000000000000000007.0.4140059104701863'}}, 'chan

In [15]:
print_checkpoint_dump(checkpointer, config)

チェックポイントデータ:
{'channel_values': {'llm_response': 'llm_response',
                    'messages': [SystemMessage(content='あなたは最小限の応答をする対話エージェントです。', additional_kwargs={}, response_metadata={}),
                                 HumanMessage(content='私の好きなものはずんだ餅です。覚えておいてね。', additional_kwargs={}, response_metadata={}),
                                 AIMessage(content='\n了解しました。', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 5, 'prompt_tokens': 40, 'total_tokens': 45, 'completion_tokens_details': None, 'prompt_tokens_details': None}, 'model_name': 'google/gemma-3-27b-it', 'system_fingerprint': None, 'id': 'gen-1745217510-nosKfuWgYCMj1aZSmKVz', 'finish_reason': 'stop', 'logprobs': None}, id='run-c7e7a504-05b3-483a-b34b-ed569d156ee8-0', usage_metadata={'input_tokens': 40, 'output_tokens': 5, 'total_tokens': 45, 'input_token_details': {}, 'output_token_details': {}}),
                                 HumanMessage(content='私の好物は何か覚えてる？', addit

In [16]:
config = {"configurable": {"thread_id": "example-2"}, "callbacks": [langfuse_handler]}
user_query = State(query="私の好物は何？")
other_thread_response = compiled_graph.invoke(user_query, config)
other_thread_response

{'query': '私の好物は何？',
 'messages': [SystemMessage(content='あなたは最小限の応答をする対話エージェントです。', additional_kwargs={}, response_metadata={}),
  HumanMessage(content='私の好物は何？', additional_kwargs={}, response_metadata={}),
  AIMessage(content='\n知りません。', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 5, 'prompt_tokens': 32, 'total_tokens': 37, 'completion_tokens_details': None, 'prompt_tokens_details': None}, 'model_name': 'google/gemma-3-27b-it', 'system_fingerprint': None, 'id': 'gen-1745217577-aAFrrUBmFJddt4ZCMCQB', 'finish_reason': 'stop', 'logprobs': None}, id='run-6ea3877b-942a-4945-b17c-c00d9080a726-0', usage_metadata={'input_tokens': 32, 'output_tokens': 5, 'total_tokens': 37, 'input_token_details': {}, 'output_token_details': {}})]}